In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns; sns.set(style="ticks", color_codes=True)
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import seaborn as sns
import cufflinks as cf
import plotly.express as px
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected = True)
cf.go_offline() 

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import mean_squared_error

In [ ]:
#Importing data
df = pd.read_csv("/kaggle/input/real-or-fake-fake-jobposting-prediction/fake_job_postings.csv")
df.head()


check the NaN Values

In [ ]:
df.isnull().values.any()

Too Many Nan-Values in data. cannot delete all of it

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

we can use sklearn.preprocessing.LabelEncoder to convert all categorial features to numeric and also consider Nan-Values as a separate category

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
def Labelencoder_feature(x):
    le=LabelEncoder()
    x=le.fit_transform(x)
    return x
df=df.apply(Labelencoder_feature)
df.head()

In [ ]:
feature=df.drop(columns=['fraudulent'])
label=df['fraudulent']

# finding the Correlation of features and target

In [ ]:
from scipy.stats import pearsonr

corre=pd.DataFrame()

for i in feature.columns:
    corre[i]= pearsonr(label, feature[i])
    
    
corre

In [ ]:
corre1=corre.T
corre1
corre1

In [ ]:
corre1=corre.T
corre1
coore2= corre1.iloc[:,0].sort_values(ascending=False)
coore2
coore2= corre1.iloc[:,0].sort_values(ascending=False)
coore2

As we can see, the first 9 features have larger correlation in compare with other features

# Reducing the dimension of data

In [ ]:
feature=df.drop(columns=['fraudulent'])
label=df['fraudulent']

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
Transf_pca = PCA(n_components= 10)
datanew = Transf_pca.fit_transform(feature)
print(datanew)
datanew.shape

In [ ]:
label=pd.DataFrame(label)
label.shape
newbase=np.concatenate((datanew,label),axis=1)
print(newbase)
newbase.shape
newbase=pd.DataFrame(newbase)
print(newbase.columns)

separating fake and real jobs and do a prediction using OneClass SVM for fake jobs

In [ ]:
dataset_selected1=newbase.loc[newbase[10].isin([1])]
dataset_selected0=newbase.loc[newbase[10].isin([0])]


#on va diviser dataset_selected1 en 2 partie data1 et label1
label1=dataset_selected1[10]
data1=dataset_selected1.drop([10],axis=1)

#on va diviser dataset_selected0 en 2 partie data0 et label0
label0=dataset_selected0[10]
#remplacer 0 par -1
label0=label0-1
data0=dataset_selected0.drop([10],axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
x_train1,x_test1,y_train1,y_test1=train_test_split(data1,label1,test_size=0.33,random_state=0)
from sklearn import svm
model=svm.OneClassSVM(kernel='rbf',nu=1,gamma=0.00001)
import time
debut=time.time()
model.fit(x_train1)
fin=time.time()-debut

In [ ]:
import numpy as np
data_tesst=np.concatenate((x_test1,data0),axis=0)
label_tesst=np.concatenate((y_test1,label0),axis=0)
pred=model.predict(data_tesst)

In [ ]:
from sklearn.metrics import accuracy_score
ACC=accuracy_score(label_tesst,pred)*100
print('the accurency score is :')
print(ACC)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(label_tesst,pred))

predict fake jobs after using PCA to reduce dimension

In [ ]:
print(newbase)
label=newbase[10]
data=newbase.drop([10],axis=1)


In [ ]:
label.value_counts()

extreme imbalanced data

Using RandomOversampling to deal with imabalanced data

In [ ]:
from imblearn.over_sampling import RandomOverSampler
import imblearn
print(imblearn.__version__)

# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy='minority')

# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy=1.0)

# fit and apply the transform
X_over, y_over = oversample.fit_resample(feature, label)
#compter combien de 1 et de 0 dans dataset
label=y_over
data=X_over
label =pd.DataFrame(label)
label.value_counts()

Applying Decision Tree Classifier to predict fakeor real posting jobs

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
dtree = DecisionTreeClassifier()
x_train,x_test,y_train,y_test = train_test_split(data,label,test_size = 0.3,random_state = 0)
import time
debut=time.time()
dtree.fit(x_train,y_train)
fin=time.time()-debut
prediction = dtree.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
ACC=accuracy_score(y_test,prediction)*100
print('With decision tree accuracy is: ',ACC) # accuracy

# Sentimental analysis

In [ ]:
#Importing data
df = pd.read_csv("/kaggle/input/real-or-fake-fake-jobposting-prediction/fake_job_postings.csv")
df.head()

In [ ]:
import warnings
warnings.filterwarnings('ignore')



from wordcloud import WordCloud , ImageColorGenerator

from nltk.corpus import stopwords
from textblob import TextBlob

In [ ]:
df=df[['description', 'fraudulent']]

df_real=df.loc[df['fraudulent']==0]
df_fake=df.loc[df['fraudulent']==1]

In [ ]:
df_fake.shape

In [ ]:
df['description'] = df['description'].astype('str')
df_fake['description'] = df_fake['description'].astype('str')

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

target_text = " ".join(df['description'])
wordcloud = WordCloud(width=1400, height=700).generate(text=target_text)
plt.figure(figsize=(30,18))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

target_text = " ".join(df_fake['description'])
wordcloud = WordCloud(width=1400, height=700).generate(text=target_text)
plt.figure(figsize=(30,18))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

target_text = " ".join(df_real['description'])
wordcloud = WordCloud(width=1400, height=700).generate(text=target_text)
plt.figure(figsize=(30,18))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
from wordcloud import WordCloud  

#for text processing
import string
import re
import nltk
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#for feature selection
from sklearn import decomposition

#for model building
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report

In [ ]:
def clean_data(text):
    text = text.lower()  # convert all the text into lowercase
    text = text.strip()  #remove starting and trailing whitespaces
    special_char_reg = '([a-zA-Z0-9]+)' + '[!"#$%&\'()*+,-./:;<=>?@\\^_`{|}~]' + '([a-zA-Z0-9]+)'
    text = re.sub(special_char_reg, ' ', text)
    text = re.sub(r'\s+', ' ', text) #remove all line formattings
    text = re.sub(r'\d+', '', text) #remove digits
    text = re.sub(r'\w*\d\w*', '', text)
    text = re.sub(r"\w+…|…", "", text)  # Remove ellipsis (and last word)
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
    text = ''.join(c for c in text if c not in string.punctuation)   #remove pecial symbols from job titles
    return text

In [ ]:
df_des=df['description']
df_des = df_des.apply(lambda x : clean_data(x))
df_des.head()

In [ ]:
from nltk import WordNetLemmatizer
def lemma(text):
    word_list = nltk.word_tokenize(text) #tokenize beofre lemmatization
    lemma_output = ' '.join(WordNetLemmatizer().lemmatize(word) for word in word_list)
    return lemma_output
df_des_lemma = df_des.apply(lambda x : lemma(x))


In [ ]:

from nltk import word_tokenize
def remove_stopwords_and_tokenize(text):
    the_stopwords = set(stopwords.words("english"))
    tokens = word_tokenize(text)  # tokenize 
    tokens = [t for t in tokens if not t in the_stopwords]  
    tokens = [t for t in tokens if len(t) > 1] 
    return tokens

In [ ]:
df_des_token = df_des_lemma.apply(lambda x : remove_stopwords_and_tokenize(x))
df_des_token.head()

In [ ]:
#Tokenization using count vectorizer
count_vect = CountVectorizer(ngram_range=(1,1))
token = count_vect.fit_transform(df_des_lemma)
token

In [ ]:
print('Total number of tokens/words in all the tweets - ', len(count_vect.get_feature_names()))

# LDA Model

In [ ]:
from gensim.corpora import Dictionary

id2word = Dictionary(df_des_token)

texts =df_des_token

corpus = [id2word.doc2bow(text) for text in texts]

corpus[:1]

In [ ]:
from gensim.models import LdaModel
lda_model = LdaModel(corpus=corpus,
                    id2word=id2word,
                    num_topics=20, 
                    random_state=100,
                    update_every=1,
                    chunksize=100,
                    passes=10,
                    alpha='auto',
                    per_word_topics=True)

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

# Forecasting

Check if we have Imbalanced Data

In [ ]:
df.isnull().sum()

In [ ]:
df['fraudulent'].value_counts()

In [ ]:
X =df['description']
y=df['fraudulent']


my_tfidf = TfidfVectorizer(stop_words='english', max_df=0.7)

# fit the vectorizer and transform X_train into a tf-idf matrix,
# then use the same vectorizer to transform X_test
tfidf_data = my_tfidf.fit_transform(X)

In [ ]:
from imblearn.over_sampling import RandomOverSampler
import imblearn
print(imblearn.__version__)

# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy='minority')

# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy=1.0)


# fit and apply the transform
X_over, y_over = oversample.fit_resample(tfidf_data, y)
#compter combien de 1 et de 0 dans dataset
label=y_over
data=X_over
label =pd.DataFrame(label)
label.value_counts()

# RandomForestClassifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.25)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
pa_rf = RandomForestClassifier()
pa_rf.fit(X_train, y_train)

In [ ]:
y_pred = pa_rf.predict(X_test)
print('ROC-AUC Score for train dataset : ' , metrics.roc_auc_score(y_test, y_pred))
print('ROC-AUC Score for validation dataset : ' , metrics.roc_auc_score(y_test, y_pred))

In [ ]:
print('Accuracy Score for train dataset : ' , metrics.accuracy_score(y_test, y_pred))
print('Accuracy Score for test dataset : ' , metrics.accuracy_score(y_test, y_pred))

In [ ]:
conn_cm_test = metrics.confusion_matrix(y_test, y_pred, [1,0])
sns.heatmap(conn_cm_test, fmt= '.2f', annot=True)